In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from mcp_use import MCPAgent, MCPClient  # Make sure to install the mcp-use package

import hud  # For telemetry

In [3]:
config = {
    "mcpServers": {
        "airbnb": {
            "command": "npx",
            "args": ["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        }
    }
}

# Create MCPClient from configuration dictionary
client = MCPClient.from_dict(config)

# Create agent with the client
agent = MCPAgent(llm=ChatOpenAI(model="gpt-4o"), client=client, max_steps=30)

In [ ]:
# Run the query inside a trace context, see it at https://app.hud.so/jobs/traces/[id]
with hud.trace(name="airbnb-test"):
    result = await agent.run(
        "Find me a nice place to stay in Barcelona for 2 adults "
        "for a week in August. I prefer places with a pool and "
        "good reviews. Show me the top 3 options.",
    )
    print(f"\nResult: {result}")

hud.flush()